<a href="https://colab.research.google.com/github/dduyg/sandbox/blob/main/pin_to_json_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


### pin_to_json_dataset.py

In [1]:
!pip install playwright pillow nest-asyncio
!playwright install chromium
!playwright install-deps

Installing dependencies...
Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Get:2 https://cli.github.com/packages stable InRelease [3,917 B]
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 3,917 B in 1s (4,304 B/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-freefont-ttf is a

In [2]:
import json
import asyncio
import time
import requests
import base64
from io import BytesIO
from PIL import Image
from playwright.async_api import async_playwright
import nest_asyncio

# Necessary for running in environments like Jupyter or specific IDEs
nest_asyncio.apply()

async def scrape_full_board_dataset(username):
    # --- CONFIGURATION ---
    RESIZE_DIM = (512, 512)

    async with async_playwright() as p:
        browser = await p.chromium.launch(headless=True)
        context = await browser.new_context(user_agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36")
        page = await context.new_page()

        print(f"[*] Accessing @{username}'s public pins...")
        await page.goto(f"https://www.pinterest.com/{username}/pins/", wait_until="domcontentloaded")

        # Hard sleep to ensure JS renders the grid
        await asyncio.sleep(5)

        # Initial scroll to trigger lazy-load
        await page.mouse.wheel(0, 2000)
        await asyncio.sleep(2)

        pins = await page.query_selector_all('div[data-test-id="pin"]')
        print(f"[*] Found {len(pins)} pins. Beginning binary extraction...")

        dataset = {
            "user": username,
            "timestamp": time.ctime(),
            "pins": []
        }

        for i, pin in enumerate(pins):
            try:
                img_tag = await pin.query_selector('img')
                if img_tag:
                    thumb_url = await img_tag.get_attribute('src')
                    alt_text = await img_tag.get_attribute('alt') or "No description"

                    # Target the original high-res image
                    high_res_url = thumb_url.replace("/236x/", "/originals/")

                    print(f"   [{i+1}/{len(pins)}] Processing: {alt_text[:30]}...")

                    # 1. Download
                    response = requests.get(high_res_url, timeout=10)
                    if response.status_code == 200:
                        # 2. Resize & Optimize in Memory
                        img = Image.open(BytesIO(response.content))
                        img = img.convert("RGB")
                        img.thumbnail(RESIZE_DIM)

                        buffer = BytesIO()
                        img.save(buffer, format="JPEG", quality=85)

                        # 3. Base64 Encode
                        img_str = base64.b64encode(buffer.getvalue()).decode('utf-8')

                        dataset["pins"].append({
                            "pin_index": i,
                            "label": alt_text,
                            "image_data": img_str,
                            "url_source": high_res_url
                        })
            except Exception as e:
                print(f"   [!] Skipped pin {i}: {e}")
                continue

        # Save to the final JSON file
        filename = f"{username}_full_dataset.json"
        with open(filename, "w", encoding="utf-8") as f:
            json.dump(dataset, f, indent=2)

        print(f"\n☑️ SUCCESS: Dataset saved to {filename}")
        print(f"🪢 Total Pins with Real Images: {len(dataset['pins'])}")
        await browser.close()

print(f"  ╔══════════════════════════════════════════════════════════════╗")
print(f"  ║                                                              ")
print(f"  ║   ██████╗ ██╗███╗   ██╗     ████████╗ ██████╗          ")
print(f"  ║   ██╔══██╗██║████╗  ██║     ╚══██╔══╝██╔═══██╗         ")
print(f"  ║   ██████╔╝██║██╔██╗ ██║        ██║   ██║   ██║          ")
print(f"  ║   ██╔═══╝ ██║██║╚██╗██║        ██║   ██║   ██║          ")
print(f"  ║   ██║     ██║██║ ╚████║        ██║   ╚██████╔╝           ")
print(f"  ║   ╚═╝     ╚═╝╚═╝  ╚═══╝        ╚═╝    ╚═════╝             ")
print(f"  ║                                                               ")
print(f"  ║            P I N  →  J S O N  →  D A T A S E T               ")
print(f"  ║                                                               ")
print(f"  ║      ◉ Cyber Scraper ◉ Pin Data Archive ◉                     ")
print(f"  ║                                                               ")
print(f"  ╚══════════════════════════════════════════════════════════════╝")

# Start the tool
user_target = input("🧞‍♀️ ENTER PINTEREST USERNAME → @ ")
await scrape_full_board_dataset(user_target)

  ╔══════════════════════════════════════════════════════════════╗
  ║                                                              
  ║   ██████╗ ██╗███╗   ██╗     ████████╗ ██████╗          
  ║   ██╔══██╗██║████╗  ██║     ╚══██╔══╝██╔═══██╗         
  ║   ██████╔╝██║██╔██╗ ██║        ██║   ██║   ██║          
  ║   ██╔═══╝ ██║██║╚██╗██║        ██║   ██║   ██║          
  ║   ██║     ██║██║ ╚████║        ██║   ╚██████╔╝           
  ║   ╚═╝     ╚═╝╚═╝  ╚═══╝        ╚═╝    ╚═════╝             
  ║                                                               
  ║            P I N  →  J S O N  →  D A T A S E T               
  ║                                                               
  ║      ◉ Cyber Scraper ◉ Pin Data Archive ◉                     
  ║                                                               
  ╚══════════════════════════════════════════════════════════════╝
🧞‍♀️ ENTER PINTEREST USERNAME → @ ghenchanaaaa
[*] Accessing @ghenchanaaaa's public pins...
[*] Fou